<a href="https://colab.research.google.com/github/GheorgheBriana/Big-data-faculty/blob/main/lab1_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


In [5]:
import findspark
findspark.init()


In [6]:
from pyspark import SparkContext # initializare spark 
import numpy as np # pentru a genera un array cu elemente aleatoare

In [7]:
sc = SparkContext(master = "local[4]")
#Spark va folosi 4 thread-uri de pe procesor "fire"

In [8]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


In [9]:
lst = np.random.randint(0, 10, 20)
#creeaza un array numpy cu 20 de numere intre 0 si 9 


In [12]:
print(lst)

[6 3 4 1 9 4 9 2 4 3 2 0 1 4 1 8 5 6 9 9]


In [14]:
# Creare RDD din array
A = sc.parallelize(lst)
# transforma array-ul numpy lst intr un RDD
# RDD-ul este o structura distribuita, imutabila care permite procesarea datelor
# pe mai multe thread-uri

In [15]:
#Afisare tip date A
A

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [16]:
A.collect()
#collect aduce toate datele din RDD inapoi intr-o lista Python
#nu se foloseste pe seturi de date mari
#un RDD nu poater fi afisat fara collect sau alte actiuni

[np.int64(6),
 np.int64(3),
 np.int64(4),
 np.int64(1),
 np.int64(9),
 np.int64(4),
 np.int64(9),
 np.int64(2),
 np.int64(4),
 np.int64(3),
 np.int64(2),
 np.int64(0),
 np.int64(1),
 np.int64(4),
 np.int64(1),
 np.int64(8),
 np.int64(5),
 np.int64(6),
 np.int64(9),
 np.int64(9)]

In [17]:
A.glom().collect()
# daca exista 20 de elem si 4 thread-uri => datele sunt impartite pe 4 partitii
# glom arata cum arata fiecare partitie/bucata. Fiecare partitie de procesata de un thread

[[np.int64(6), np.int64(3), np.int64(4), np.int64(1), np.int64(9)],
 [np.int64(4), np.int64(9), np.int64(2), np.int64(4), np.int64(3)],
 [np.int64(2), np.int64(0), np.int64(1), np.int64(4), np.int64(1)],
 [np.int64(8), np.int64(5), np.int64(6), np.int64(9), np.int64(9)]]

In [18]:
#Numarare elemente din RDD
A.count()

20

In [19]:
A.first()
#primul element din RDD

np.int64(6)

In [20]:
A.take(4)
#returneaza primele 4 elemente

[np.int64(6), np.int64(3), np.int64(4), np.int64(1)]

In [22]:
A_distinct = A.distinct()
#distinct creeaza un nou RDD fara duplicate (nu il afiseaza inca)
# originalul (A) ramane neschimbat

In [23]:
A_distinct.collect()

[np.int64(4),
 np.int64(0),
 np.int64(8),
 np.int64(1),
 np.int64(9),
 np.int64(5),
 np.int64(6),
 np.int64(2),
 np.int64(3)]

In [26]:
A.reduce(lambda x, y: x + y)
#reduce - combina toate elementele din RDD folosind o functie data (aici: suma)

np.int64(90)

In [27]:
A.sum()

np.int64(90)

In [28]:
A.fold(0, lambda x, y: x + y)
#similara cu reduce, dar porneste de la o valoare data (aici: 0)

np.int64(90)

In [33]:
A.reduce(lambda x, y: x if x > y else y)
#se va afisa cel mai mare nr
#logica este asta: se vor compara toate elementele dintr o partitie, 2 cate 2 => se gaseste cel mai mare element local/de pe acel rand
# se face un array (imaginar) cu toți acești maximi locali si se alege cel mai mare nr din acel array/ cel mai mare numar global

np.int64(9)

In [34]:
words = 'These are some of the best Macintosh computers ever'.split(' ')
#creeaza o lista de cuvinte impartind propozitia la fiecare spatiu
#['These', 'are', 'some', 'of', 'the', 'best', 'Macintosh', 'computers', 'ever']

In [37]:
A.filter(lambda x: x % 3 == 0).collect()
#filter pastreaza toate elem din RDD care respecta conditia data
#collect aduce rezultatul sub forma de lista

[np.int64(6),
 np.int64(3),
 np.int64(9),
 np.int64(9),
 np.int64(3),
 np.int64(0),
 np.int64(6),
 np.int64(9),
 np.int64(9)]

In [45]:
#fctie personalizata pt reduce pe cuvinte
def largerThan(x, y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x) > len(y): #daca x e mai lung => return x
      return x
    elif len(y) > len(x): #daca y e mai lung => return y
      return y
    else: #daca sunt egale alfabetic
      if x < y: #daca x vine inaintea lui y alfabetic => return x, else return t
        return x
      else:
        return y


In [43]:
wordRDD = sc.parallelize(words)
#transformam words intr un RDD

In [48]:
wordRDD.reduce(largerThan)
#Macinsoth si computers au ambele cele mai multe caractere (9)
#returneaza Macintosh deoarece M < c. De ce? pt ca in ASCII: M = 77, c = 99

'Macintosh'

In [47]:
words

['These', 'are', 'some', 'of', 'the', 'best', 'Macintosh', 'computers', 'ever']

In [50]:
#Sampling, esantionare aleatorie din RDD
"""
RDD.sample(withReplacement, p)
----------------------------------
sample(withReplacement, fraction)
-
with replacement = TRUE/ FALSE
TRUE - un elem poate fi ales de mai multe ori
FALSE - un elem este ales doar o sg data
-
fraction, adesea notat cu p:
Probabilitatea ca fiecare elem sa fie inclus in esantion
ex: fraction = 0.5 => Spark incearca sa aleaga 50% din elem (dar nu garanteaza exact)
"""
m = 5
n = 20
print('sample1=', A.sample(False, m/n).collect())
#False, fara inlocuire, un elem e ales doar o data
#m/n reprez probabilitatea ca fiecare sa fie ales, in acest caz 5/20 => 0.25
print('sample2=', A.sample(False, m/n).collect())
#!!!FIECARE ELM ARE 25 % SA FIE ALES

sample1= [np.int64(4), np.int64(9), np.int64(2), np.int64(0), np.int64(8), np.int64(6), np.int64(9)]
sample2= [np.int64(3), np.int64(4), np.int64(9), np.int64(1), np.int64(6)]


In [57]:
print(A.collect())
#Statistici de baza
#max
print("Maximum: ", A.max())
#min
print("Minimum:", A.min())
#media artimetica
print("Mean(average): ", A.mean())
#deviatia standard
print("Standard deviation: ", A.stdev())

[np.int64(6), np.int64(3), np.int64(4), np.int64(1), np.int64(9), np.int64(4), np.int64(9), np.int64(2), np.int64(4), np.int64(3), np.int64(2), np.int64(0), np.int64(1), np.int64(4), np.int64(1), np.int64(8), np.int64(5), np.int64(6), np.int64(9), np.int64(9)]
Maximum:  9
Minimum: 0
Mean(average):  4.5
Standard deviation:  2.9410882339705484


In [58]:
A.stats()

(count: 20, mean: 4.5, stdev: 2.9410882339705484, max: 9.0, min: 0.0)

In [60]:
#Mapare simpla
B = A.map(lambda x: x * x)
#Creare nou RDD, B.
#Fiecare elem din A este inlocuit de patratul sau (X*X in B)
#Map ia fiecare elemnt din RDD si il transforma dupa o anumita conditie spusa de noi

In [61]:
B.collect()

[np.int64(36),
 np.int64(9),
 np.int64(16),
 np.int64(1),
 np.int64(81),
 np.int64(16),
 np.int64(81),
 np.int64(4),
 np.int64(16),
 np.int64(9),
 np.int64(4),
 np.int64(0),
 np.int64(1),
 np.int64(16),
 np.int64(1),
 np.int64(64),
 np.int64(25),
 np.int64(36),
 np.int64(81),
 np.int64(81)]

In [64]:
def square_if_odd(x):
  if x % 2 == 1: # x impar => returnare x * x
    return x * x
  else: # x par => returnare x
    return x

In [70]:
#Aplicare fctie custom
A.map(square_if_odd).collect()

[np.int64(6),
 np.int64(9),
 np.int64(4),
 np.int64(1),
 np.int64(81),
 np.int64(4),
 np.int64(81),
 np.int64(2),
 np.int64(4),
 np.int64(9),
 np.int64(2),
 np.int64(0),
 np.int64(1),
 np.int64(4),
 np.int64(1),
 np.int64(8),
 np.int64(25),
 np.int64(6),
 np.int64(81),
 np.int64(81)]

In [66]:
A.flatMap(lambda x:(x, x*x)).collect()
#flatMap -> aplica o functie care returneaza o lista/tuplu de elemente pentru fiecare element RDD
#in aces caz, fiecare element x produce 2 valori: x si x*x
#diferenta fata de map este ca: flatmap poate returna mai multe elemente, iar map returneaza
# un singur element per intrare
# EX: Dacă A are [2, 3], rezultatul e [2, 4, 3, 9].

[np.int64(6),
 np.int64(36),
 np.int64(3),
 np.int64(9),
 np.int64(4),
 np.int64(16),
 np.int64(1),
 np.int64(1),
 np.int64(9),
 np.int64(81),
 np.int64(4),
 np.int64(16),
 np.int64(9),
 np.int64(81),
 np.int64(2),
 np.int64(4),
 np.int64(4),
 np.int64(16),
 np.int64(3),
 np.int64(9),
 np.int64(2),
 np.int64(4),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(4),
 np.int64(16),
 np.int64(1),
 np.int64(1),
 np.int64(8),
 np.int64(64),
 np.int64(5),
 np.int64(25),
 np.int64(6),
 np.int64(36),
 np.int64(9),
 np.int64(81),
 np.int64(9),
 np.int64(81)]

In [71]:
#Grupari si bining
result = A.groupBy(lambda x:x%2).collect()
#grupeaza elementele din RDD dupa o cheie(fctie data)
#x%2 => paritate; 0:pt nr pare, 1:pt impare
#Dacă RDD-ul A are [1, 2, 3, 4, 5]:
#   Grupa 0 (pare) → [2, 4]; Grupa 1 (impare) → [1, 3, 5]


sorted([(x, sorted(y)) for (x, y) in result])
#sortare
# x -> cheia (0, 1); y -> grupul de elemente

[(np.int64(0),
  [np.int64(0),
   np.int64(2),
   np.int64(2),
   np.int64(4),
   np.int64(4),
   np.int64(4),
   np.int64(4),
   np.int64(6),
   np.int64(6),
   np.int64(8)]),
 (np.int64(1),
  [np.int64(1),
   np.int64(1),
   np.int64(1),
   np.int64(3),
   np.int64(3),
   np.int64(5),
   np.int64(9),
   np.int64(9),
   np.int64(9),
   np.int64(9)])]

In [73]:
B.histogram([x for x in range(0, 100, 10)])
# creeaza intervale si numara cate elemente sunt in fiecare interval

"""
 Prima listă - intervalele (buckets):
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Asta înseamnă că sunt 9 intervale:

[0-10), [10-20), [20-30), ..., [80-90).

Intervalele sunt deschise la dreapta:

[0-10) - include 0 și până la 9.999... (nu include 10).
"""

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [8, 4, 1, 2, 0, 0, 1, 0, 4])

In [81]:
#cartesian, produsul carteziar dintre 2 RDD-uri
lst1 = np.random.randint(0, 10, 3)
C = sc.parallelize(lst1)

lst2 = np.random.randint(10, 20, 3)
D = sc.parallelize(lst2)

print("C", C.collect())
print("D", D.collect())

(C+D).collect()
#unire cele 2 RDD-URI

C.cartesian(D).collect()
#Produsul cartezian, combina fiecare elem din C cu fiecare elem din D


C [np.int64(1), np.int64(0), np.int64(3)]
D [np.int64(16), np.int64(14), np.int64(10)]


[(np.int64(1), np.int64(16)),
 (np.int64(1), np.int64(14)),
 (np.int64(1), np.int64(10)),
 (np.int64(0), np.int64(16)),
 (np.int64(0), np.int64(14)),
 (np.int64(0), np.int64(10)),
 (np.int64(3), np.int64(16)),
 (np.int64(3), np.int64(14)),
 (np.int64(3), np.int64(10))]

In [84]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()
#intersectarea a 2 RDD-uri

[1, 2, 3]

In [87]:
rdd1.subtract(rdd2).collect()
#ce este rdd1 si nu in rdd2

[10, 4, 5]

In [88]:
sc.stop()